# Noteboook for testing view classification

In [ ]:
import sys
import os
import pandas as pd
import datetime
import numpy as np

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir, 'src'))

from d00_utils.db_utils import dbReadWriteClassification, dbReadWriteViews
import d02_intermediate.download_dcm as download_dcm
from d02_intermediate.dcm_utils import dcmdir_to_jpgs_for_classification
import d03_classification.predict_views as predict_views


import random

from d00_utils.db_utils import dbReadWriteViews
from d00_utils.s3_utils import download_s3_objects
from d02_intermediate.dcm_utils import decompress_dcm
from d00_utils.log_utils import setup_logging
logger = setup_logging(__name__, __name__)


In [ ]:
downsample_ratio = 0.05
train_test_ratio = 0
table_name = 'instances_w_labels_train'
train = False

In [ ]:
dcm_dir_suffix = download_dcm.s3_download_decomp_dcm(table_name, train_test_ratio, downsample_ratio, train)

In [ ]:
dcm_dir = os.path.join(os.path.expanduser('~/data/01_raw/'), dcm_dir_suffix)
img_dir = os.path.join(os.path.expanduser('~/data/02_intermediate/'), dcm_dir_suffix)
#dcmdir_to_jpgs_for_classification(dcm_dir, img_dir)

In [ ]:
model_path = '/home/ubuntu/models/view_23_e5_class_11-Mar-2018'
feature_dim = 1
if_exists='replace'
predictions = predict_views.run_classify(img_dir, model_path, if_exists, feature_dim)

In [ ]:
io_classification = dbReadWriteClassification()
io_classification.list_tables()

In [ ]:
df = io_classification.get_table('predictions')
df.head()

In [ ]:
predict_views.agg_predictions()

In [ ]:
#to overwrite existing predictions table
#io_classification.save_to_db(df.iloc[:0,:-1], "predictions", if_exists="replace")

In [ ]:
io_classification.raw_conn.rollback() #to reset after error